You may find this series of notebooks at [databricks-solutions/realtime-rag-agents-databricks-youcom](https://github.com/databricks-solutions/realtime-rag-agents-databricks-youcom). For more information about this solution accelerator, visit the [blog post](https://you.com/articles/unlocking-real-time-intelligence-for-ai-agents-with-you.com-and-databricks).

In [0]:
%pip install -U -qqqq mlflow-skinny[databricks]>=3.1.0 langgraph==0.3.4 databricks-langchain databricks-agents uv gspread google-auth
dbutils.library.restartPython()

In [0]:
import pandas as pd
import pyspark.sql.functions as F
import io
import yaml

# Agent Configs

1. Create a [Connection](https://docs.databricks.com/aws/en/connect/) to access the You.com Search API. 

    * This connection definition assumes that You.com API key is persisted as a Databricks [Secret](https://docs.databricks.com/aws/en/security/secrets/).

2. Create a [UC Function](https://docs.databricks.com/aws/en/udf/unity-catalog) that makes an [http_request](https://docs.databricks.com/aws/en/sql/language-manual/functions/http_request) using the connection to perform search for user's query.


## 0. Initial Configurations

This notebook accepts parameters via Databricks job widgets when run as part of a job, or falls back to `common.yaml` for interactive development.

**For Interactive Development**: Create a `common.yaml` file with your configuration values.

**For Job Execution**: Parameters are passed automatically via the job configuration.

In [ ]:
# Optional: Create common.yaml for interactive development
# Uncomment and run this cell if you're developing interactively (not running as a job)

# %%writefile common.yaml
# catalog: # TODO: Fill in with UC Catalog
# schema: # TODO: Fill in with UC Schema
# secret_scope: # TODO: Fill in with Secret Scope where You.com API key
# secret_key: # TODO: Fill in with Secret Key for You.com API key

## 1. Load Configurations

In [ ]:
# Set up widgets to accept job parameters
dbutils.widgets.text("catalog", "main", "UC Catalog")
dbutils.widgets.text("schema", "default", "UC Schema")
dbutils.widgets.text("secret_scope", "", "Secret Scope")
dbutils.widgets.text("secret_key", "", "Secret Key")

# Get widget values
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
secret_scope = dbutils.widgets.get("secret_scope")
secret_key = dbutils.widgets.get("secret_key")

# If widget values are empty/default, try loading from common.yaml as fallback
if not secret_scope or not secret_key:
    try:
        import yaml
        cfg = yaml.safe_load(open("common.yaml"))
        catalog = catalog if catalog != "main" else cfg.get("catalog", "main")
        schema = schema if schema != "default" else cfg.get("schema", "default")
        secret_scope = secret_scope or cfg.get("secret_scope", "")
        secret_key = secret_key or cfg.get("secret_key", "")
    except FileNotFoundError:
        pass  # No fallback file, use widget values

# Validate configuration
assert secret_scope, "Missing secret_scope. Provide via widget or common.yaml"
assert secret_key, "Missing secret_key. Provide via widget or common.yaml"

function_name = "you_com_search_function"
function_path = f"{catalog}.{schema}.{function_name}"
connection = "you_com_connection"

print(f"Configuration loaded:")
print(f"  Catalog: {catalog}")
print(f"  Schema: {schema}")
print(f"  Secret Scope: {secret_scope}")
print(f"  Function Path: {function_path}")

## 2. Create Connection and UC Function

In [0]:
host = "https://api.ydc-index.io"
port = 443
api_path = "/v2/search"
sql_query = f"""CREATE CONNECTION IF NOT EXISTS {connection}
TYPE HTTP
OPTIONS (
  host '{host}',
  port {port},
  base_path '{api_path}',
  bearer_token secret('{secret_scope}', '{secret_key}')
)"""

spark.sql(sql_query)

In [0]:
sql_query = f"""CREATE OR REPLACE FUNCTION {function_path}(query STRING)
RETURNS STRING
COMMENT 'Search for information relative to the query'
RETURN (
  http_request(
    conn => "{connection}",
    method => "GET",
    path => concat('?query=', url_encode(query))
  ).text
)"""
print(sql_query)
spark.sql(sql_query)

## [OPTIONAL] 3. Test UC Function 

In [ ]:
df = spark.createDataFrame(pd.DataFrame({"query": ["What are the current flight delays at San Francisco International Airport?"]}))

display(df.withColumn("search_results", F.expr(f"{function_path}(query)")))